In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-23"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
527,2024-07-23,Canadá Cebl,21:00,Winnipeg Sea Bears,Calgary Surge,1.94,1.73,178.5,1.83,1.83,0.0,0.00,0.00,ANLhRAye,0.515464,0.578035,0.546448,0.546448,0.093499,187.702,19.185594,0.102213,1.2,1.643168,1.369306,213.44,2.040,0.179861,0.088167,-8.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,92,105,2.32,1.98,0.000,0.000,0.080922,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
528,2024-07-23,Mundo Amistosos Internacionais,13:00,Austrália F,Canadá F,1.22,3.98,139.5,1.83,1.87,0.0,0.00,0.00,2iBGEcN9,0.819672,0.251256,0.546448,0.534759,0.070928,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,75,0,1.33,0.00,0.000,0.000,0.750621,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
529,2024-07-23,México Lnbp,23:00,Diablos Rojos,Santos,1.37,2.93,168.5,1.85,1.85,0.0,0.00,0.00,fHI6NHJk,0.729927,0.341297,0.540541,0.540541,0.071224,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,193.564,13.852806,0.071567,1.2,1.643168,1.369306,203.28,2.414,0.543075,0.224969,-8.0,0,66,0.00,3.08,0.000,0.000,0.513064,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
530,2024-07-23,República Dominicana Lnb,21:00,Titanes Del Licey,Metros de Santiago,1.68,2.10,172.5,1.84,1.84,0.0,0.00,0.00,pQsmOb8F,0.595238,0.476190,0.543478,0.543478,0.071429,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,232.666,73.361566,0.315308,1.8,1.643168,0.912871,164.34,2.468,0.805028,0.326186,21.0,88,99,1.76,1.66,0.000,0.000,0.157135,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
531,2024-07-23,Porto Rico Bsn,21:00,Aguada Santeros,Criollos de Caguas,1.60,2.24,173.5,1.82,1.88,0.0,0.00,0.00,dEOm3Qnd,0.625000,0.446429,0.549451,0.531915,0.071429,173.656,43.546048,0.250760,1.2,1.643168,1.369306,182.78,2.088,0.496105,0.237598,-6.0,154.060,25.670424,0.166626,1.8,1.643168,0.912871,136.50,1.846,0.461660,0.250087,6.0,74,91,2.47,1.50,161.372,178.786,0.235702,0.022933,NaN,0.12,0.024,25.000000,0.589240,0.7,0.110760,-0.73,-0.146,-8.493151,0.475548,0.4,-0.075548
532,2024-07-23,Mundo Amistosos Internacionais,16:00,EUA F,Alemanha F,1.01,12.20,155.5,1.86,1.84,0.0,0.00,0.00,0zV64X0L,0.990099,0.081967,0.537634,0.543478,0.072066,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,54,0.00,1.01,0.000,0.000,1.197960,0.007644,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
533,2024-07-23,Mundo Amistoso Internacional,15:30,Espanha,Porto Rico,1.39,2.84,164.5,1.86,1.84,0.0,0.00,0.00,dfxh5Yfo,0.719424,0.352113,0.537634,0.543478,0.071537,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,76,93,1.28,9.45,0.000,0.000,0.484778,0.007644,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
534,2024-07-23,Colômbia Lbp,22:05,Titanes de Barranquilla,Toros del Valle,1.03,9.55,165.5,1.85,1.85,0.0,0.00,0.00,x

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
